In [1]:
import pandas as pd
import re
import string
import nltk
import ssl
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
import pandas as pd
!pip install openpyxl

file_path = 'C:/Users/hj00m/OneDrive - 금오공과대학교/3. Lab/2. 김민준_교수님/2. data/ROBLOX_DATA_PS.xlsx'

# 엑셀 파일 읽기
df = pd.read_excel(file_path, engine='openpyxl')

# 데이터프레임 출력
print(df.head())  # 처음 5개 행 출력

                               reviewId             userName  \
0  8b33921b-b971-49ad-9c43-5f46535d8481       Rhylenn Taylor   
1  ddfa4628-ecd7-403d-b4f8-976655986017      Addalaide Clark   
2  8d9d55dd-df25-4329-af58-2178007c5434  jennavie kajornklin   
3  493e1a17-3af8-4e60-b5d2-a1f9f1056355              Josilyn   
4  4ac6c4f5-c019-46f0-b829-8e98d19a93ea                  Eli   

                                             content  score  thumbsUpCount  \
0  Perfect App! Though, There are a few problems ...      5            609   
1  I love this game. I have so much fun playing w...      4           1384   
2  Roblox is my favorite game. But when I play on...      4            576   
3  This game is great, amazing almost. But my sit...      1            166   
4  been glitching and completely freezing in the ...      1            143   

  reviewCreatedVersion                  at appVersion  
0            2.578.564 2023-06-24 03:07:01  2.578.564  
1            2.578.564 2023-06-25 

In [3]:
duplicates = df[df.duplicated(subset=['reviewId', 'at', 'content', 'score'], keep=False)]
print(f"중복된 행 수: {len(duplicates)}")

if not duplicates.empty:
    df = df.drop_duplicates(subset=['reviewId', 'at', 'content', 'score'], keep='first')
    print("중복된 행을 제거했습니다.")
else:
    print("중복된 행이 없습니다.")

중복된 행 수: 0
중복된 행이 없습니다.


In [4]:
# NLTK 다운로드 설정
try:
    _create_unverified_https_context = ssl._create_unverified_context
    ssl._create_default_https_context = _create_unverified_https_context
except AttributeError:
    pass

In [5]:
# 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# 전처리
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'
    elif tag.startswith('V'):
        return 'v'
    elif tag.startswith('N'):
        return 'n'
    elif tag.startswith('R'):
        return 'r'
    else:
        return 'n'

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if not isinstance(text, str):
        return []
    text = text.lower()
    text = remove_emoji(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tagged_tokens = pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]
    return lemmatized


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hj00m\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hj00m\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hj00m\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hj00m\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [ ]:
# 토큰화
df['tokens'] = df['content'].apply(preprocess_text)
df['token_count'] = df['tokens'].apply(len)
df_filtered = df[df['token_count'] >= 10].copy()

print(df_filtered.head())

In [ ]:
# LDA
from gensim import corpora
from gensim.models import LdaModel

texts = df_filtered['tokens'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=9,
                     passes=10,
                     random_state=42)


In [ ]:
# 키워드 추출
topic_keywords = []
for i in range(9):
    words = lda_model.show_topic(i, topn=15)
    keywords = [word for word, _ in words]
    topic_keywords.append(keywords)

for idx, kw in enumerate(topic_keywords):
    print(f"Topic {idx+1} keywords:", kw)


Topic 1 keywords: ['account', 'get', 'roblox', 'say', 'log', 'try', 'back', 'cant', 'update', 'work', 'go', 'let', 'please', 'password', 'delete']
Topic 2 keywords: ['robux', 'game', 'get', 'buy', 'like', 'money', 'free', 'dont', 'need', 'thing', 'make', 'wish', 'good', 'love', 'pay']
Topic 3 keywords: ['game', 'like', 'people', 'say', 'dont', 'good', 'get', 'bad', 'thing', 'really', 'roblox', 'fun', 'hacker', 'word', 'ban']
Topic 4 keywords: ['kid', 'great', 'game', 'roblox', 'app', 'good', 'would', 'child', 'think', 'age', 'amaze', 'recommend', 'way', 'make', 'use']
Topic 5 keywords: ['na', 'add', 'wan', 'remove', 'gon', 'pls', 'back', 'please', 'plz', 'outfit', 'bring', 'skin', 'hair', 'premium', 'jailbreak']
Topic 6 keywords: ['game', 'play', 'fun', 'love', 'friend', 'like', 'roblox', 'make', 'best', 'really', 'much', 'many', 'get', 'ever', 'app']
Topic 7 keywords: ['game', 'fix', 'play', 'roblox', 'good', 'please', 'cant', 'lag', 'like', 'bug', 'problem', 'phone', 'sometimes', 'wo

키워드 유사어 확장

In [ ]:
import re
from gensim.models import Word2Vec

def remove_emojis_from_tokens(tokens):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002700-\U000027BF"
        u"\U000024C2-\U0001F251"
        u"\U0001F900-\U0001F9FF"
        "]+", flags=re.UNICODE)
    return [t for t in tokens if not emoji_pattern.search(t)]

# df_filtered['tokens'] = df_filtered['tokens'].apply(remove_emojis_from_tokens)

# Word2Vec 학습
model_w2v = Word2Vec(sentences=df_filtered['tokens'],
                     vector_size=100, window=5, min_count=5,
                     workers=4, sg=1)

# 키워드 확장
expanded_topic_keywords = []

for topic in topic_keywords:
    expanded_words = set(topic)
    for word in topic:
        if word in model_w2v.wv:
            similar_words = model_w2v.wv.most_similar(word, topn=3)
            for similar_word, score in similar_words:
                expanded_words.add(similar_word)
    # 최대 47개까지만 유지 -> 가장 작은 값이 47개라
    expanded_topic_keywords.append(list(expanded_words)[:47])

for i, words in enumerate(expanded_topic_keywords):
    print(f"Topic {i+1} 확장 키워드 ({len(words)}개): {words}")


Topic 1 확장 키워드 (47개): ['cant', 'accout', 'guyss', 'updates', 'account', 'butwhen', 'git', 'exited', 'workbut', 'get', 'barley', 'allow', 'backand', 'plz', 'pasword', 'deleted', 'updatei', 'woke', 'sign', 'deleting', 'please', 'updating', 'work', 'laging', 'kindof', 'login', 'happenedi', 'couldnt', 'fricken', 'worked', 'back', 'password', 'return', 'tryin', 'delete', 'passwordi', 'gat', 'pls', 'incorrect', 'log', 'game', 'deleate', 'go', 'logout', 'lem', 'roblox', 'init']
Topic 2 확장 키워드 (47개): ['rproleplay', 'gamess', 'idont', 'wondering', 'somethings', 'could', 'loveee', 'create', 'iove', 'mony', 'shouldent', 'need', 'tycons', 'better', 'git', 'fun', 'lods', 'moneyand', 'get', 'thing', 'buy', 'love', 'cuter', 'like', 'roux', 'cast', 'buying', 'good', 'robix', 'robuxs', 'cheeper', 'multilayer', 'kindof', 'freebut', 'moneybut', 'wish', 'cost', 'needs', 'stuff', 'nead', 'friendsmeet', 'gat', 'money', 'stuffbut', 'game', 'great', 'dont']
Topic 3 확장 키워드 (47개): ['itsometimes', 'harassment', 

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
analyzer = SentimentIntensityAnalyzer()

df = df_filtered.copy()
df['content'] = df['content'].astype(str).str.lower()

# 리뷰별 감성 점수 벡터 생성
review_feature_scores = []
for text in df['content']:
    row_scores = []
    for keywords in expanded_topic_keywords:
        if any(kw in text for kw in keywords):
            score = analyzer.polarity_scores(text)['compound']
        else:
            score = 0.0
        row_scores.append(round(score, 2)) 
    review_feature_scores.append(row_scores)

features_df = pd.DataFrame(review_feature_scores, columns=[f'f{i+1}' for i in range(9)])
features_df.insert(0, "No.", range(1, len(features_df) + 1))

# 평점 이진화
df = df.reset_index(drop=True)
features_df = features_df.reset_index(drop=True)
features_df['Overall Rating'] = df['score'].apply(lambda x: 1 if x >= 3 else 0)

print(features_df.head())

In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

X = features_df[[f"f{i+1}" for i in range(9)]]
X = sm.add_constant(X) 
y = features_df["Overall Rating"]

model = sm.Logit(y, X)
result = model.fit()

odds_ratios = np.exp(result.params)
conf_int = result.conf_int()
conf_int_exp = np.exp(conf_int)
summary_df = pd.DataFrame({
    "Coefficient": result.params,
    "Odds Ratio": odds_ratios,
    "CI Lower": conf_int_exp[0],
    "CI Upper": conf_int_exp[1],
    "P-value": result.pvalues
})

summary_df = summary_df.round(4)
summary_df.index.name = "Variable"

print(summary_df)